In [ ]:
import pandas as pd
import numpy as np

import teradata
import calendar
import time
import unicodedata
import math
from collections import defaultdict

##############################################################################
try:
    user = input("Teradata Username: ") #  # Teradata Username
    pswd = input("Teradata Password: ") #  # Teradata Password
    db = input("Database ")   #"db"
except NameError: #Python 3
    pass
print("user: ", user, "pswd: ", pswd, "db: ",db)
##############################################################################

SourceTableName = #"db.sTable" # name of the table with the source data to be mapped
NewTableName = #"db.dTable" # name of the table to create and store results

In [ ]:
###Teradata Functions and queries



### Opening a Teradata Connection
def openTeradataConnection(db, user, pswd):
    udaExec = teradata.UdaExec(appName="TestConnection", version="0.1", logConsole=False)
    session = udaExec.connect(method="odbc", system=db, username=user, password=pswd)
    return session

### Executing a query
def executeQuery(session, query):
    print("Executing query...")
    startTime = calendar.timegm(time.gmtime())
    queryData = session.execute(query)
    qtime = calendar.timegm(time.gmtime())
    print ("Query took ", str(qtime - startTime), " seconds.")
    return queryData

###create table
def createTable(NewTableName):
    executeQuery(session, """create multiset table """+NewTableName+""" ( ind varchar(50), val varchar(50)
             ) primary index(ind)""")

###insert values into table
def uploadToTdTable(tableName, data):
    max_batchsize = 16383
    num_batches = math.ceil(len(data)/max_batchsize)
    for i in range(num_batches):
        upload = data[max_batchsize*(i):max_batchsize*(i+1)]
        session.executemany("""INSERT INTO """+tableName+""" (ind, val) VALUES (?, ?)""", upload, batch=True)
        print("finished uploading batch",i+1, "out of",num_batches)
    print("done!")

### Convert source data to dictionary
def sourceToDict(df):
    d = defaultdict(list)
    for i, j in zip(df.a,df.b):
        d[i].append(j)
    for i in d:
        d[i].append(i)
    return d


### Check if value exists in defaultdict
def in_values(s, d):
    """Does `s` appear in any of the values in `d`?"""
    for v in d.values():
        if s in v:
            return True
    return False



### BFS to make groups dictionary (instead of sourceToDict())


        
def makeGroups(pairs):      
    def step(temp):
        while len(temp) > 0:
            x = temp.pop(0)
            for b in pairs[pairs.a == x].b:
                if b not in temp and b != x and not in_values(b, done):
                    temp.append(b)
                    bList.remove(b)
            done[i].append(x)
    bList = list(pd.unique(pairs[['a', 'b']].values.ravel()))
    temp = []
    done = defaultdict(list)
    i = 1
    while len(bList) > 0:
        temp.append(bList.pop(0))
        step(temp)
        i+=1
    return done


### Teradata query
TdQuery = """select * from """+SourceTableName

In [ ]:
#run queries
session = openTeradataConnection(db, user, pswd)
data = executeQuery(session, TdQuery)


In [ ]:
#convert to DataFrame
df = pd.DataFrame(data.fetchall())
df.columns = ['a','b']

testdf = pd.DataFrame([['a','b'],['a','c'],['a','d'],['b','d'],['b','e'],['d','f'],['e','g'],['h','i'],['h','j'],['i','j'],['k','a']])
testdf.columns = ['a','b']
testdf

In [ ]:
#create groups dictionary
d = makeGroups(df)

In [ ]:
mapping = pd.DataFrame([[k,vv]  for k,v in d.items() for vv in v])
data = [tuple(x) for x in mapping.to_records(index=False)]

In [ ]:
# create table and upload mapping

#createTable(NewTableName)
#uploadToTdTable(NewTableName, data)

